In [1]:
# Import Necessary Libraries
from dotenv import load_dotenv, dotenv_values
import google.generativeai as genai
from IPython.display import Markdown, display
import os

# Load .env file

load_dotenv()

# Set the TESSDATA_PREFIX environment variable

tessdata = os.getenv("TESSDATA_PREFIX")
os.environ['TESSDATA_PREFIX'] = tessdata

# Set the Google api key

my_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=my_api_key)

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings,GoogleGenerativeAI

llm = GoogleGenerativeAI(model ="gemini-pro",temperature=0)
embedding = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") 
model = ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [3]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["oracle_sql", "mysql", "mongodb"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
structured_llm = model.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
chain = prompt | structured_llm




In [4]:
from langchain_core.prompts import PromptTemplate

oracle_chain = PromptTemplate.from_template(
    """You are an expert in oracle database. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)
mysql_chain = PromptTemplate.from_template(
    """You are an expert in mysql database. \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)
mongodb_chain = PromptTemplate.from_template(
    """You are an expert in mongodb. \
Respond to the following question:
Question: {question}
Answer:"""
) | ChatGoogleGenerativeAI(model= "gemini-1.5-flash", temperature = 0)

In [5]:
def choose_route(question):
    result = chain.invoke({"question": question})
    if "oracle_sql" in result.datasource.lower():
        ### Logic here 
        print("Using Oracle")
        return oracle_chain
    elif "mysql" in result.datasource.lower():
        ### Logic here 
        print("Using mysql")
        return mysql_chain
    else:
        ### Logic here 
        print("Using mongodb")
        return mongodb_chain

from langchain_core.runnables import RunnableLambda

#full_chain = router | RunnableLambda(choose_route)

In [6]:

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(
    choose_route) | prompt

In [7]:


result = full_chain.invoke({"question": "Write a stored procedure to concatenate first name and last name in mysql"})
print(result)

Using mysql
messages=[SystemMessage(content='You are an expert at routing a user question to the appropriate data source.\n\nBased on the programming language the question is referring to, route it to the relevant data source.'), HumanMessage(content='content="```sql\\nDELIMITER //\\n\\nCREATE PROCEDURE ConcatenateNames(IN userId INT, OUT fullName VARCHAR(255))\\nBEGIN\\n  SELECT CONCAT(firstName, \' \', lastName) INTO fullName\\n  FROM users\\n  WHERE id = userId;\\nEND //\\n\\nDELIMITER ;\\n```\\n\\n**Explanation:**\\n\\n1. **DELIMITER //:** This line changes the default delimiter from `;` to `//` to allow the stored procedure definition to span multiple lines.\\n2. **CREATE PROCEDURE ConcatenateNames(IN userId INT, OUT fullName VARCHAR(255)):** This line defines the stored procedure named `ConcatenateNames`. It takes an input parameter `userId` of type `INT` and an output parameter `fullName` of type `VARCHAR(255)`.\\n3. **BEGIN:** This line marks the start of the stored procedure\'

In [8]:
result = full_chain.invoke({"question": "Write a query to select all the documents from a MongoDB database named Employees"})
print(result)

Using mongodb
messages=[SystemMessage(content='You are an expert at routing a user question to the appropriate data source.\n\nBased on the programming language the question is referring to, route it to the relevant data source.'), HumanMessage(content='content=\'```javascript\\ndb.Employees.find({});\\n```\\n\\n**Explanation:**\\n\\n* `db.Employees` refers to the "Employees" collection within your MongoDB database.\\n* `.find({})` is the MongoDB command to retrieve documents from a collection.\\n* `{}` is an empty document filter, which means it will match all documents in the collection.\\n\\nThis query will return all documents from the "Employees" collection. \\n\' response_metadata={\'prompt_feedback\': {\'block_reason\': 0, \'safety_ratings\': []}, \'finish_reason\': \'STOP\', \'safety_ratings\': [{\'category\': \'HARM_CATEGORY_SEXUALLY_EXPLICIT\', \'probability\': \'NEGLIGIBLE\', \'blocked\': False}, {\'category\': \'HARM_CATEGORY_HATE_SPEECH\', \'probability\': \'NEGLIGIBLE\', 